# DATA FOUNDATION

In [10]:
import pandas as pd
import numpy as np 
import os
import gc # Để giải phóng RAM

BASE_DIR = "/MALLORN-Astronomical-Classification-Challenge/data/raw"

R_COEFFS = {'u': 4.239, 'g': 3.303, 'r': 2.285, 'i': 1.698, 'z': 1.263, 'y': 1.086}

In [15]:
def load_train_log(base_dir):
    train_log_path  = os.path.join(base_dir, 'train_log.csv')
    if not os.path.exists(train_log_path):
        raise FileNotFoundError("Không tìm thấy file train_log")
    
    df_train_log = pd.read_csv(train_log_path)
    return df_train_log

In [19]:
def apply_sanity_checks(df):
    n_original = len(df)

    # 1. Xứ lý NaN
    # Nếu Flux hoặc Flux_err bị NaN -> Xóa dòng vì dữ liệu vô nghĩa
    df = df.dropna(subset=['Flux', 'Flux_err', 'Flux_corrected', 'Flux_err_corrected'])

    # Nếu EBV bị NaN, điền bằng 0 (coi như không có bụi) để tránh lỗi tính toán
    if 'EBV' in df.columns:
        df['EBV'] = df['EBV'].fillna(0)

    # 2. Xử lý Flux âm
    # Không xóa FLux âm nhưng kiểm kê nó
    n_negative = (df['Flux_corrected'] < 0).sum()

    # 3. Sắp xếp thời gian
    df = df.sort_values(by=['object_id', 'Time (MJD)'], ascending=[True, True])
    df = df.reset_index(drop=True)

    n_dropped = n_original - len(df)
    if n_dropped > 0:
        print(f"   Đã loại bỏ {n_dropped} dòng chứa NaN.")
    if n_negative > 0:
        print(f"   Lưu ý: Có {n_negative} điểm đo có Flux Âm (vẫn giữ lại).")
    
    return df

In [20]:
def process_one_split(split_name, df_train_log, base_dir):
    print(f"\n Đang xử lý: {split_name}...")
    
    train_lc_path = os.path.join(base_dir, split_name, "train_full_lightcurves.csv")
    
    df_meta_split = df_train_log[df_train_log['split'] == split_name].copy()
    
    df_train_lc = pd.read_csv(train_lc_path)

    df_train_lc_merged = df_train_lc.merge(
        df_meta_split[['object_id', 'EBV', 'target']],
        on='object_id',
        how='inner'
    )

    df_train_lc_merged['R_factor'] = df_train_lc_merged['Filter'].map(R_COEFFS)
    correction = 10 ** (0.4 * df_train_lc_merged['R_factor'] * df_train_lc_merged['EBV'])
    df_train_lc_merged['Flux_corrected'] = df_train_lc_merged['Flux'] * correction
    df_train_lc_merged['Flux_err_corrected'] = df_train_lc_merged['Flux_err'] * correction

    df_train_lc_clean = apply_sanity_checks(df_train_lc_merged)
    del df_train_lc, df_train_lc_merged
    gc.collect()

    print(f"   -> Hoàn tất {split_name}")
    return df_train_lc_clean

In [26]:
df_log_global = load_train_log(BASE_DIR)

# Test trên split_01
df_result = process_one_split('split_01', df_log_global, BASE_DIR)

if df_result is not None:
    print("\n--- KIỂM TRA DỮ LIỆU CUỐI CÙNG (5 dòng đầu) ---")
    # Kiểm tra xem thời gian đã tăng dần chưa
    display(df_result[['object_id', 'Time (MJD)', 'Flux', 'Flux_corrected']].head(10))
    
    # Kiểm tra thống kê NaN
    print("\nSố lượng NaN còn lại (phải là 0):")
    print(df_result.isna().sum())


 Đang xử lý: split_01...
   Đã loại bỏ 11 dòng chứa NaN.
   Lưu ý: Có 10254 điểm đo có Flux Âm (vẫn giữ lại).
   -> Hoàn tất split_01

--- KIỂM TRA DỮ LIỆU CUỐI CÙNG (5 dòng đầu) ---


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6728\3041852526.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EBV'] = df['EBV'].fillna(0)


,object_id,Time (MJD),Flux,Flux_corrected
0,Dornhoth_fervain_onodrim,63314.4662,-1.630159,-1.852686
1,Dornhoth_fervain_onodrim,63314.4662,-1.424537,-1.590222
2,Dornhoth_fervain_onodrim,63327.6691,-1.409011,-1.673503
3,Dornhoth_fervain_onodrim,63327.6691,-1.558067,-1.770754
4,Dornhoth_fervain_onodrim,63340.8720,-2.057437,-2.443647
5,Dornhoth_fervain_onodrim,63380.4806,-0.855074,-1.194911
6,Dornhoth_fervain_onodrim,63380.4806,-1.021596,-1.287713
7,Dornhoth_fervain_onodrim,63420.0891,-1.073603,-1.353268
8,Dornhoth_fervain_onodrim,63420.0891,-0.472944,-0.660909
9,Dornhoth_fervain_onodrim,63424.4901,-1.080999,-1.283918



Số lượng NaN còn lại (phải là 0):
object_id             0
Time (MJD)            0
Flux                  0
Flux_err              0
Filter                0
EBV                   0
target                0
R_factor              0
Flux_corrected        0
Flux_err_corrected    0
dtype: int64


# FEATURE ENGINEERING

STATISTICAL FEATURES

In [24]:
def generate_statistical_features(df):
    print("Đang tạo: Statistical & Percentiles Features....")

    # 1. Định nghĩa các hàm thống kê
    aggregations = {
        'Flux_corrected': [
            'mean', 'std', 'max', 'min',
            ('q05', lambda x: x.quantile(0.05)), # Phân vị 5% (thay cho Min để tránh nhiễu)
            ('q25', lambda x: x.quantile(0.25)),
            ('q75', lambda x: x.quantile(0.75)),
            ('q95', lambda x: x.quantile(0.95)), # Phân vị 95% (thay cho Max để tránh nhiễu)
            'skew', # Độ lệch phân phối (quan trọng cho TDE)
            'count' # Số lượng điểm đo
        ],
        'Flux': ['max'], # Giữ lại max flux gốc để tham chiếu
        'Flux_err': ['mean'] # Sai số trung bình (đánh giá chất lượng đo)
    }

    # 2. Groupby và Aggregation (Thực hiện song song cho tất cả object)
    # Biến đổi bảng Long-fomr thành bảng thống kê sơ bộ
    df_agg = df.groupby(['object_id', 'Filter']).agg(aggregations)

    # 3. Làm phảng MultiIndex Columns
    df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

    # 4. Unstack (Xoay trục Filter thành cột)
    df_features = df_agg.unstack('Filter')
    df_features.columns = [f"{filter_name}_{feature}" for feature, filter_name in df_features.columns]

    # 5. Tính các đặc trưng "Global" (Tổng hợp trên mọi band)
    global_agg = df.groupby('object_id')['Flux_corrected'].agg([
        ('global_max', 'max'),
        ('global_mean', 'mean'),
        ('global_std', 'std')
    ])

    # Ghép global vào bảng features
    df_features = df_features.join(global_agg)

    # 6. Xử lý NaN sinh ra do Unstack
    # (Ví dụ: Object A không có dữ liệu filter 'u', sẽ sinh ra NaN tại các cột u_...)
    # Chiến thuật: 
    # - Với Mean, Max, Min, Quantile: Fill = 0 (coi như tối đen)
    # - Với Count: Fill = 0
    df_features = df_features.fillna(0)

    print(f"✅ Hoàn thành Nhóm 1! Kích thước: {df_features.shape}")
    return df_features

In [27]:
df_features_g1 = generate_statistical_features(df_result)

# Xem thử 5 cột đầu tiên để kiểm tra định dạng tên
print("\n--- Mẫu các đặc trưng vừa tạo ---")
display(df_features_g1.iloc[:5, :10]) # Xem 10 cột đầu

# Kiểm tra xem có cột skew nào (độ lệch)
skew_cols = [c for c in df_features_g1.columns if 'skew' in c]
print(f"\nĐã tạo {len(skew_cols)} cột Skewness (ví dụ: {skew_cols[0]})")

Đang tạo: Statistical & Percentiles Features....
✅ Hoàn thành Nhóm 1! Kích thước: (155, 75)

--- Mẫu các đặc trưng vừa tạo ---


,g_Flux_corrected_mean,i_Flux_corrected_mean,r_Flux_corrected_mean,u_Flux_corrected_mean,y_Flux_corrected_mean,z_Flux_corrected_mean,g_Flux_corrected_std,i_Flux_corrected_std,r_Flux_corrected_std,u_Flux_corrected_std
object_id,,,,,,,,,,
Dornhoth_fervain_onodrim,-0.523170,2.507891,1.354417,0.897305,-0.448582,1.442674,0.949155,7.521850,4.080687,1.756665
Dornhoth_galadh_ylf,0.214970,0.387165,0.310346,0.033881,0.748245,0.563584,0.432050,1.059712,0.742890,0.707609
Elrim_melethril_thul,3.141294,6.419646,4.381228,0.057594,0.058178,7.847825,2.504693,7.115146,3.270901,3.117768
Ithil_tobas_rodwen,0.288862,0.454172,0.445994,0.160318,0.347413,0.538618,0.362132,0.640980,0.536924,0.462454
Mirion_adar_Druadan,0.055118,0.420397,0.236805,-0.017642,0.305273,0.316992,0.959285,1.478422,1.218722,0.530510



Đã tạo 6 cột Skewness (ví dụ: g_Flux_corrected_skew)


MORPHOLOGY FEATURES

In [76]:
def generate_morphology_features(df_clean, df_features_g1):
    print("   Đang tạo: Morphology Features")

    # index của dòng có Flux lớn nhất cho mỗi object/filter
    idx_max = df_clean.groupby(['object_id', 'Filter'])['Flux_corrected'].idxmax()

    # index của dòng đầu tiên và dòng cuối cùng (theo thời gian)
    idx_first = df_clean.groupby(['object_id', 'Filter'])['Time (MJD)'].idxmin()
    idx_last = df_clean.groupby(['object_id', 'Filter'])['Time (MJD)'].idxmax()

     # Trích xuất dữ liệu tại các điểm mốc
    df_peaks = df_clean.loc[idx_max][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])
    df_starts = df_clean.loc[idx_first][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])
    df_ends = df_clean.loc[idx_last][['object_id', 'Filter', 'Time (MJD)', 'Flux_corrected']].set_index(['object_id', 'Filter'])

    df_peaks.columns = ['Time_peak', 'Flux_peak']
    df_starts.columns = ['Time_start', 'Flux_start']
    df_ends.columns = ['Time_end', 'Flux_end']

    # Ghép lại thành một bảng cho morphology
    df_morp = pd.concat([df_peaks, df_starts, df_ends], axis=1)

    # Tính Time, Slope
    df_morp['Rise_time'] = df_morp['Time_peak'] - df_morp['Time_start']
    df_morp['Fall_time'] = df_morp['Time_end'] - df_morp['Time_peak']
    df_morp['Rise_slope'] = (df_morp['Flux_peak'] - df_morp['Flux_start']) / (df_morp['Rise_time'] + 0.1)
    df_morp['Fall_slope'] = (df_morp['Flux_end'] - df_morp['Flux_peak']) / (df_morp['Fall_time'] + 0.1)
    
    cols_time_slope = ['Rise_time', 'Fall_time', 'Rise_slope', 'Fall_slope']
    df_morp_ts = df_morp[cols_time_slope].unstack('Filter')
    df_morp_ts.columns = [f"{filter_name}_{feat}" for feat, filter_name in df_morp_ts.columns]

    # Tính Percent Amplitude
    amp_features = []
    filters = ['u', 'g', 'r', 'i', 'z', 'y']
    
    for f in filters:
        col_max = f"{f}_Flux_corrected_max"
        col_min = f"{f}_Flux_corrected_min"
        col_mean = f"{f}_Flux_corrected_mean"

        amp_series = (df_features_g1[col_max] - df_features_g1[col_min]) / (df_features_g1[col_mean] + 1e-6)
        amp_df = pd.DataFrame({f"{f}_percent_amplitude": amp_series})
        amp_features.append(amp_df)
    df_morp_amp = pd.concat(amp_features, axis=1)

    # Tính Kurtosis
    df_morp_kurt = df_clean.groupby(['object_id', 'Filter'])['Flux_corrected'].apply(lambda x: x.kurt()).unstack('Filter')
    df_morp_kurt.columns = [f"{f}_kurtosis" for f in df_morp_kurt.columns]

    # Tính Stetson K
    mean_cols = [c for c in df_features_g1.columns if 'mean' in c and 'Flux_corrected' in c and 'global' not in c]
    df_means = df_features_g1[mean_cols].copy()
    df_means.columns = [c.split('_')[0] for c in df_means.columns]

    df_means_flat = df_means.stack().reset_index()
    df_means_flat.columns = ['object_id', 'Filter', 'mean_flux']
    df_stetson = pd.merge(df_clean, df_means_flat, on=['object_id', 'Filter'], how='left')

    df_stetson['delta'] = (df_stetson['Flux_corrected'] - df_stetson['mean_flux']) / (df_stetson['Flux_err_corrected'] + 1e-6)
    
    df_stetson['abs_delta'] = df_stetson['delta'].abs()
    df_stetson['delta_sq'] = df_stetson['delta'] ** 2

    g = df_stetson.groupby(['object_id', 'Filter'])
    mean_abs_delta = g['abs_delta'].mean()
    mean_delta_sq = g['delta_sq'].mean()
    stetson_k = mean_abs_delta / (np.sqrt(mean_delta_sq) + 1e-6)

    df_morp_stetson = stetson_k.unstack('Filter')
    df_morp_stetson.columns = [f"{f}_stetson_k" for f in df_morp_stetson.columns]


    # Tạo bảng morphology features
    df_morp_final = df_morp_ts.join([df_morp_amp, df_morp_kurt, df_morp_stetson])

    # Xử lý NaN nếu có
    df_morp_final = df_morp_final.fillna(0)

    print(f"Hoàn thành Nhóm 2! Kích thước: {df_morp_final.shape}")
    return df_morp_final

In [78]:
df_features_g2 = generate_morphology_features(df_result, df_features_g1)

print("\n--- Mẫu đặc trưng Hình thái ---")
display(df_features_g2.iloc[:5, :8]) # Xem 8 cột đầu

   Đang tạo: Morphology Features
Hoàn thành Nhóm 2! Kích thước: (155, 42)

--- Mẫu đặc trưng Hình thái ---


,g_Rise_time,i_Rise_time,r_Rise_time,u_Rise_time,y_Rise_time,z_Rise_time,g_Fall_time,i_Fall_time
object_id,,,,,,,,
Dornhoth_fervain_onodrim,444.4963,444.4964,400.4868,61.6134,475.3031,457.6993,250.8544,699.7516
Dornhoth_galadh_ylf,1504.5906,1473.7800,1453.2394,682.9712,1558.5094,1537.9690,785.6737,798.5114
Elrim_melethril_thul,0.0000,730.9880,0.0000,444.6663,767.8628,620.3637,498.8939,472.8647
Ithil_tobas_rodwen,1119.8792,1119.8792,2451.7049,2781.0811,2726.6625,1094.1019,1729.9413,1729.9413
Mirion_adar_Druadan,762.0820,1104.4667,996.2289,713.4855,1153.0632,629.5460,1422.5531,1080.1684


COLOR FEATURES

In [84]:
def generate_color_features(df_features_g1):
    print("   Đang tạo: Color Features")

    filters = ['u', 'g', 'r', 'i', 'z', 'y']
    new_features = []

    for i in range(len(filters) - 1):
        f1 = filters[i]
        f2 = filters[i+1]

        col_f1_mean = f"{f1}_Flux_corrected_mean"
        col_f2_mean = f"{f2}_Flux_corrected_mean"

        ratio_mean = df_features_g1[col_f1_mean] / (df_features_g1[col_f2_mean] + 1e-6)
        diff_mean = df_features_g1[col_f1_mean] - df_features_g1[col_f2_mean]

        col_f1_max = f"{f1}_Flux_corrected_max"
        col_f2_max = f"{f2}_Flux_corrected_max"

        ratio_max = df_features_g1[col_f1_max] / (df_features_g1[col_f2_max] + 1e-6)
        diff_max = df_features_g1[col_f1_max] - df_features_g1[col_f2_max]

        temp_df = pd.DataFrame({
            f'{f1}_{f2}_mean_ratio': ratio_mean,
            f'{f1}_{f2}_mean_diff': diff_mean,
            f'{f1}_{f2}_max_ratio': ratio_max,
            f'{f1}_{f2}_max_diff': diff_max
        }, index=df_features_g1.index)

        new_features.append(temp_df)

    f_start, f_end = 'u', 'y'
    col_start = f"{f_start}_Flux_corrected_mean"
    col_end = f"{f_end}_Flux_corrected_mean"

    blue_red_ratio = df_features_g1[col_start] / (df_features_g1[col_end] + 1e-6)
    temp_extreme = pd.DataFrame({f'{f_start}_{f_end}_mean_ratio': blue_red_ratio}, index=df_features_g1.index)
    new_features.append(temp_extreme)

    df_colors = pd.concat(new_features, axis=1)
    df_colors = df_colors.replace([np.inf, -np.inf], 0).fillna(0)

    print(f"Hoàn thành Nhóm 3! Kích thước: {df_colors.shape}")
    return df_colors


In [85]:
df_features_g3 = generate_color_features(df_features_g1)
    
print("\n--- Mẫu đặc trưng Màu sắc (u/g, g/r...) ---")
display(df_features_g3.iloc[:5, :5]) 

   Đang tạo: Color Features
Hoàn thành Nhóm 3! Kích thước: (155, 21)

--- Mẫu đặc trưng Màu sắc (u/g, g/r...) ---


,u_g_mean_ratio,u_g_mean_diff,u_g_max_ratio,u_g_max_diff,g_r_mean_ratio
object_id,,,,,
Dornhoth_fervain_onodrim,-1.715132,1.420475,2.932089,2.658946,-0.386269
Dornhoth_galadh_ylf,0.157608,-0.181089,0.891903,-0.161195,0.692675
Elrim_melethril_thul,0.018334,-3.083700,0.481356,-3.768502,0.716989
Ithil_tobas_rodwen,0.554996,-0.128544,1.186430,0.264779,0.647682
Mirion_adar_Druadan,-0.320078,-0.072761,0.376521,-1.101521,0.232757
